# Automated Model Deployment as a RESTful Prediction Service on Kubernetes

The purpose of this notebook is to demonstrate how the Machine Learning (ML) model generated as a 'build artefact' of the  `titanic-ml.ipynb` notebook in this project's root directory, can be automatically deployed as a fully-managed RESTful service on Kubernetes using the scripted commands contained within this notebook. The required steps:

1. configure a Flask REST API wrapper for an ML model using a customised `.env` file;
2. build a Docker image for the API and push it to an image registry;
3. deploy an end-to-end managed service to a Kubernetes cluster using a custom Helm chart; and,
4. test that the service is working.

## Package Imports

In [1]:
import os

import docker

## Deployment Configuration Parameters

In [3]:
MODEL_REPOSITORY = '../models'
SERVICE_NAME = 'titanic'
API_VERSION = '1'
DOCKER_IMAGE_REGISTRY = 'cmftall'

## Copy Model to Flask Service Source Code Directory

Which is `py-sklearn-flask-ml-service` within the current directory.

In [5]:
latest_model = sorted(os.listdir(MODEL_REPOSITORY))[-1]
os.popen(f'cp {latest_model} deploy/py-sklearn-flask-ml-service/model.joblib')
print(f'{latest_model} --> deploy/py-sklearn-flask-ml-service/model.joblib')

titanic-ml-2019-06-03T17:18:04.joblib --> deploy/py-sklearn-flask-ml-service/model.joblib


## Write Service Configuration Parameters

We use a `.env` file in `py-sklearn-flask-ml-service` that will be copied to the Docker image, automatically loaded by Pipenv and used by the Flask service described in `py-sklearn-flask-ml-service` to define the REST API endpoint.

In [6]:
with open('py-sklearn-flask-ml-service/.env', 'w') as file:
    file.writelines(f'SERVICE_NAME={SERVICE_NAME}\n')
    file.writelines(f'API_VERSION={API_VERSION}\n')

Validate the contents of the `.env` file.

In [7]:
!cat py-sklearn-flask-ml-service/.env

SERVICE_NAME=titanic
API_VERSION=1


## Build Docker Image

Ensure that there is a Docker daemon up-and-runnning.

> Note, in production we will need a build server (e.g. Travis CI) to build images and push to registries.

In [8]:
image_tag = f'{DOCKER_IMAGE_REGISTRY}/{SERVICE_NAME}:latest'

docker_client = docker.from_env()
image, build_log = docker_client.images.build(
    path='py-sklearn-flask-ml-service', tag=image_tag, rm=True)

docker_client.images.prune()

for line in build_log:
    print(line)

{'stream': 'Step 1/7 : FROM python:3.7-slim'}
{'stream': '\n'}
{'status': 'Pulling from library/python', 'id': '3.7-slim'}
{'status': 'Already exists', 'progressDetail': {}, 'id': '743f2d6c1f65'}
{'status': 'Already exists', 'progressDetail': {}, 'id': '977e13fc7449'}
{'status': 'Pulling fs layer', 'progressDetail': {}, 'id': 'de5f9e5af26b'}
{'status': 'Pulling fs layer', 'progressDetail': {}, 'id': '0d27ddbe8383'}
{'status': 'Pulling fs layer', 'progressDetail': {}, 'id': '228d55eb5a23'}
{'status': 'Downloading', 'progressDetail': {'current': 240, 'total': 240}, 'progress': '[==================================================>]     240B/240B', 'id': '0d27ddbe8383'}
{'status': 'Verifying Checksum', 'progressDetail': {}, 'id': '0d27ddbe8383'}
{'status': 'Download complete', 'progressDetail': {}, 'id': '0d27ddbe8383'}
{'status': 'Downloading', 'progressDetail': {'current': 22401, 'total': 2104020}, 'progress': '[>                                                  ]   22.4kB/2.104MB', 'id'

## Push Image to DockerHub

- Will need to use `docker_client.login()` on a build server.

In [10]:
push_log = docker_client.images.push(image_tag)
print(push_log)

{"status":"The push refers to repository [docker.io/cmftall/titanic]"}
{"status":"Preparing","progressDetail":{},"id":"bd6cb57a91d0"}
{"status":"Preparing","progressDetail":{},"id":"5fa9cc4411e6"}
{"status":"Preparing","progressDetail":{},"id":"25cb58f60f3b"}
{"status":"Preparing","progressDetail":{},"id":"e4d8d737bd10"}
{"status":"Preparing","progressDetail":{},"id":"d7f161fd8308"}
{"status":"Preparing","progressDetail":{},"id":"e6b8bc0a67bc"}
{"status":"Preparing","progressDetail":{},"id":"2ca9f4d774f4"}
{"status":"Preparing","progressDetail":{},"id":"c56a33c38007"}
{"status":"Waiting","progressDetail":{},"id":"e6b8bc0a67bc"}
{"status":"Waiting","progressDetail":{},"id":"2ca9f4d774f4"}
{"status":"Preparing","progressDetail":{},"id":"6270adb5794c"}
{"status":"Waiting","progressDetail":{},"id":"6270adb5794c"}
{"status":"Layer already exists","progressDetail":{},"id":"25cb58f60f3b"}
{"status":"Layer already exists","progressDetail":{},"id":"e4d8d737bd10"}
{"status":"Layer already exists

## Deploy to Kubernetes using Helm Charts

We use the Helm chart located at `k8s-helm-ml-prediction-app` and discussed [here](https://github.com/AlexIoannides/kubernetes-ml-ops) to deploy our ML REST API as a fully-managed, self-healing and load balanced service on Kubernetes.

Note, ensure that `kubectl config current-context` is set the Kubernetes cluster we wish to deploy to and that this cluster has an operational Helm Tiller.

In [12]:
!helm install k8s-helm-ml-prediction-app \
    --set app.name="$SERVICE_NAME-survival-prediction" \
    --set app.namespace="$SERVICE_NAME" \
    --set app.image="$image_tag"

NAME:   imprecise-frog
LAST DEPLOYED: Mon Jun  3 21:33:22 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Namespace
NAME     STATUS  AGE
titanic  Active  1s

==> v1/Pod(related)
NAME                                  READY  STATUS             RESTARTS  AGE
titanic-survival-prediction-rc-9glv5  0/1    ContainerCreating  0         1s
titanic-survival-prediction-rc-d5wfj  0/1    Pending            0         1s

==> v1/ReplicationController
NAME                            DESIRED  CURRENT  READY  AGE
titanic-survival-prediction-rc  2        2        0      1s

==> v1/Service
NAME                            TYPE          CLUSTER-IP     EXTERNAL-IP  PORT(S)         AGE
titanic-survival-prediction-lb  LoadBalancer  10.102.198.72  <pending>    5000:30372/TCP  1s


NOTES:
Thank you for installing helm-ml-score-app.

Your release is named imprecise-frog.

To learn more about the release, try:

  $ helm status imprecise-frog
  $ helm get imprecise-frog



## Test Prediction Service

We will assume that Minikube is being used for local testing, in which case we will need to ask it for the URL of its 'virtual load balancer' used for our service.

In [13]:
!minikube service list

|-------------|--------------------------------|-----------------------------|
|  NAMESPACE  |              NAME              |             URL             |
|-------------|--------------------------------|-----------------------------|
| default     | kubernetes                     | No node port                |
| kube-system | kube-dns                       | No node port                |
| kube-system | kubernetes-dashboard           | No node port                |
| kube-system | tiller-deploy                  | No node port                |
| titanic     | titanic-survival-prediction-lb | http://192.168.99.102:30372 |
|-------------|--------------------------------|-----------------------------|


We can then take the appropriate URL from the above and test our titanic prediction service!

In [18]:
!curl http://192.168.99.102:30372/titanic/v1/predict \
        --request POST \
        --header 'Content-Type: application/json' \
        --data '{"Pclass": [1], "Sex": ["male"], "Age": [32], "SibSp": [1],                      "Parch": [0], "Fare": [100], "Embarked": ["S"]}'

{"prediction":[1]}
